In [13]:
#Read in KDD99 Data Set¶
import pandas as pd

df1 = pd.read_csv('Churn_Modelling1.csv')

print("Read {} rows.".format(len(df1)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset

df1.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)


# display 5 rows
df1[0:5]

Read 10000 rows.


,CustomerId,Surname,CreditScore,Exited
0,15634602,Hargrave,619,1
1,15647311,Hill,608,0
2,15619304,Onio,502,1
3,15701354,Boni,699,0
4,15737888,Mitchell,850,0


In [14]:
ENCODING = 'utf-8'

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [15]:
# Analyze KDD-99

import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

analyze(df1)


10000 rows
** CustomerId:10000 (100%)
** Surname:2932 (29%)
** CreditScore:460 (4%)
** Exited:[0:79.63%,1:20.37%]


In [16]:
#df = df1.drop('protocol_type', axis=1)

# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

    
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [17]:
encode_numeric_zscore(df1, 'CustomerId')
encode_numeric_zscore(df1, 'CreditScore')

In [18]:
df1[0:5]

,CustomerId,Surname,CreditScore,Exited
0,-0.783174,Hargrave,-0.326205,1
1,-0.606504,Hill,-0.440014,0
2,-0.995835,Onio,-1.536717,1
3,0.144759,Boni,0.501496,0
4,0.652626,Mitchell,2.063781,0


In [28]:
df1=df1.drop('Surname',axis=1)

In [29]:
# Convert to numpy - Classification
x_columns1 = df1.columns.drop('Exited')
x1 = df1[x_columns1].values
dummies1 = pd.get_dummies(df1['Exited']) # Classification
outcomes1 = dummies1.columns
num_classes1 = len(outcomes1)
y1 = dummies1.values

In [30]:
df1.groupby('Exited')['Exited'].count()

Exited
0    7963
1    2037
Name: Exited, dtype: int64

In [31]:
from sklearn.model_selection import train_test_split

x1_train, x1_test, y1_train, y1_test = train_test_split(
    x1, y1, test_size=0.25, random_state=42)

In [33]:
print(x1)

[[-0.78317426 -0.32620511]
 [-0.60650379 -0.44001395]
 [-0.99583497 -1.53671734]
 ...
 [-1.47920782  0.60495814]
 [-0.1193498   1.25677242]
 [-0.87051556  1.46369759]]


In [34]:


from sklearn import metrics
from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(25, input_dim=x1.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(25, activation='relu'))

model.add(Dense(x1.shape[1], activation='linear'))(original_branch) 
model.compile(loss='mean_squared_error', optimizer='adam')

################ SAVING THE LAST OUTPUT LAYER ##################

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense

# Assuming you have already defined your original model as 'model'
# and it looks like the one in your previous code

# Create a new model with just the last layer
last_layer = Dense(x1.shape[1], name='output_layer')(model.layers[-1].output)
output_model = Model(inputs=model.input, outputs=last_layer)

# Save the last layer model to a file
output_model.save('last_layer_model.h5')


################## ATTENTION WEIGHTS ########################

attention_weights = AttentionLayer()(original_branch)


# Merge the attention weights with the new input features
merged_input = Concatenate()([attention_weights, input_new])



In [35]:

# Convert the output into a Pandas dataframe
pd1 = pd.DataFrame(x1, columns=['C_output_neuron_{}'.format(i+1) for i in range(x1.shape[1])])

# Save the output to a CSV file
pd1.to_csv('C_dense_output1.csv', index=False)

In [36]:
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(10, input_dim=x1.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x1.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x1.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y1.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x1_train,y1_train,validation_data=(x1_test,y1_test),callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
235/235 - 1s - loss: 0.5658 - val_loss: 0.5007 - 541ms/epoch - 2ms/step
Epoch 2/1000
235/235 - 0s - loss: 0.5087 - val_loss: 0.4995 - 185ms/epoch - 789us/step
Epoch 3/1000
235/235 - 0s - loss: 0.5085 - val_loss: 0.5001 - 189ms/epoch - 806us/step
Epoch 4/1000
235/235 - 0s - loss: 0.5080 - val_loss: 0.4990 - 185ms/epoch - 789us/step
Epoch 5/1000
235/235 - 0s - loss: 0.5087 - val_loss: 0.4991 - 188ms/epoch - 802us/step
Epoch 6/1000
235/235 - 0s - loss: 0.5083 - val_loss: 0.4998 - 188ms/epoch - 802us/step
Epoch 7/1000
235/235 - 0s - loss: 0.5080 - val_loss: 0.4985 - 187ms/epoch - 795us/step
Epoch 7: early stopping


In [37]:
import numpy as np

# Measure accuracy
pred = model.predict(x1_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y1_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

79/79 [==============================] - 0s 510us/step
Validation score: 0.8012
